# Market Basket Analysis

Based on https://github.com/chris1610/pbpython/blob/master/notebooks/Market_Basket_Intro.ipynb

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_style('darkgrid')
random_state = 42
pd.set_option('display.float_format', lambda x: '%.3f' % x)

%matplotlib inline

In [11]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Prepare the data (skip if already prepared)

In [3]:
%%time

hours = ['0'+str(x)+':00' if x < 10 else str(x)+':00' for x in range(24)]
hour_type = pd.CategoricalDtype(categories=hours, ordered=True)

dtype={'GUESTCHECKID': object,
       'Date': str,
       'HourName': hour_type,
       'QuarterName': "category",
       'GUESTCHECK_SalesNet': np.float64,
       'GUESTCHECK_SalesTax': np.float64,
       'Product': "category",
       'FamilyGroup': "category",
       'MajorGroup': "category",
       'Product_SalesNet': np.float64,
       'Product_SalesTax': np.float64,
       'MPK': object,
       'Restaurant': object,
       'LocationType': "category",
       'Concept': "category",
       'ItemType': "category",
       'ComboMealNum': np.float64,
       'ile_razy': np.float64,
       'SalesChannel': "category"
       }
parse_dates = ['Date']

data = pd.read_csv(os.path.join(os.environ['DATA_PATH'], 'kiosk_produkty/KIOSK_Produkty.csv'), delimiter=";", thousands=',', error_bad_lines=False, dtype=dtype, parse_dates=parse_dates)

CPU times: user 44.5 s, sys: 4.76 s, total: 49.3 s
Wall time: 49.7 s


### Create basket table

In [4]:
# https://github.com/pandas-dev/pandas/issues/19136#issuecomment-380908428
def reset_index(df):
    '''Returns DataFrame with index as columns'''
    index_df = df.index.to_frame(index=False)
    df = df.reset_index(drop=True)
    #  In merge is important the order in which you pass the dataframes
    # if the index contains a Categorical. 
    # pd.merge(df, index_df, left_index=True, right_index=True) does not work
    return pd.merge(index_df, df, left_index=True, right_index=True)

In [5]:
%%time
basket = (data.groupby(['GUESTCHECKID', 'Product'])['ile_razy']
          .sum().unstack())

CPU times: user 16.1 s, sys: 6.56 s, total: 22.7 s
Wall time: 22.7 s


In [ ]:
# Delete unnecesary data and save memory
del data

In [ ]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [6]:
%%time
basket = reset_index(basket).fillna(0).set_index('GUESTCHECKID').applymap(encode_units)

CPU times: user 36.5 s, sys: 16.2 s, total: 52.7 s
Wall time: 53 s


In [15]:
%%time
basket_sets = basket.applymap(encode_units)

CPU times: user 8min 51s, sys: 31.8 s, total: 9min 23s
Wall time: 9min 25s


In [ ]:
basket_sets.head()

In [ ]:
%%time
basket_sets.to_csv(os.path.join(os.environ['DATA_PATH'], 'basket_sets/basket_sets.csv'))

## Load data (if prepared before)

In [34]:
%%time
basket_sets = pd.read_csv(os.path.join(os.environ['DATA_PATH'], 'basket_sets/basket_sets.csv'), index_col=0)

CPU times: user 2min 36s, sys: 9.36 s, total: 2min 45s
Wall time: 2min 46s


In [17]:
basket_sets.head()

,10HotWings,10Strips,10xCOBHS,10xCOBKent,10xHotWings,10xStripsHS,11HotWings,11Strips,12xHotWings,12xStripsHS,...,Zinger_wege,app_2xCheesburg,app_2xSmallCoff,app_Bites&Fries,Halloumi,TwisteHaluBox,TwisteHalum,mHalloumi,mHalloumiBox,mTwisteHalum
GUESTCHECKID,,,,,,,,,,,,,,,,,,,,,
1000000246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000000247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000000406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000000415,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000000416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Basket analysis

In [18]:
%%time
# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

CPU times: user 57 s, sys: 21.9 s, total: 1min 18s
Wall time: 1min 19s


In [19]:
frequent_itemsets.head()

,support,itemsets
0,0.097,(2HotWings)
1,0.119,(2Strips)
2,0.091,(5HotWings)
3,0.998,(Customer)
4,0.077,(DipMajoGarlic)


In [20]:
%%time
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=5)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 8.03 ms


#### Remove standard sets - related associations
If someone buys Bsmart then we expect fries or sandwitch to be in the basket too. This associations are not interesting so we remove them.

In [21]:
len(rules)

504

In [31]:
rows_w_bsmart = (rules['antecedents'].astype(str).str.contains('Bsmart', regex=False) | rules['consequents'].astype(str).str.contains('Bsmart', regex=False))
rows_w_app_bucketfor1 = (rules['antecedents'].astype(str).str.contains('app_bucketfor1', regex=False) | rules['consequents'].astype(str).str.contains('app_bucketfor1', regex=False))
fillter_sets_out = (~rows_w_bsmart & ~rows_w_app_bucketfor1)

In [32]:
len(rules[fillter_sets_out])

36

In [40]:
pd.set_option('display.max_rows', 200)
rules[~rows_w_bsmart & ~rows_w_app_bucketfor1].sort_values('lift', ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
98,"(DrumstickKent., 2HotWings)","(Fries, ex_45BitesAddon)",0.072,0.083,0.070,0.985,11.927,0.065,62.047
93,"(Fries, ex_45BitesAddon)","(DrumstickKent., 2HotWings)",0.083,0.072,0.070,0.853,11.927,0.065,6.318
99,(ex_45BitesAddon),"(Fries, DrumstickKent., 2HotWings)",0.084,0.071,0.070,0.839,11.897,0.065,5.764
92,"(Fries, DrumstickKent., 2HotWings)",(ex_45BitesAddon),0.071,0.084,0.070,0.999,11.897,0.065,1737.724
30,"(DrumstickKent., 2HotWings)",(ex_45BitesAddon),0.072,0.084,0.071,0.999,11.895,0.065,1410.298
31,(ex_45BitesAddon),"(DrumstickKent., 2HotWings)",0.084,0.072,0.071,0.851,11.895,0.065,6.222
97,"(Fries, 2HotWings)","(DrumstickKent., ex_45BitesAddon)",0.085,0.071,0.070,0.827,11.565,0.064,5.356
94,"(DrumstickKent., ex_45BitesAddon)","(Fries, 2HotWings)",0.071,0.085,0.070,0.986,11.565,0.064,64.670
40,(ex_45BitesAddon),"(Fries, 2HotWings)",0.084,0.085,0.083,0.983,11.535,0.075,54.562
39,"(Fries, 2HotWings)",(ex_45BitesAddon),0.085,0.084,0.083,0.969,11.535,0.075,29.614


In [39]:
rules.sort_values('lift', ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
67,"(Fries, iTwistB)",(mMiniTwistBsmart),0.075,0.075,0.073,0.971,12.919,0.067,31.416
68,(mMiniTwistBsmart),"(Fries, iTwistB)",0.075,0.075,0.073,0.967,12.919,0.067,27.871
73,(mLongerBsmart),"(Fries, Longer)",0.075,0.077,0.073,0.970,12.573,0.067,31.125
70,"(Fries, Longer)",(mLongerBsmart),0.077,0.075,0.073,0.949,12.573,0.067,18.025
69,(iTwistB),"(mMiniTwistBsmart, Fries)",0.082,0.073,0.073,0.889,12.168,0.067,8.331
66,"(mMiniTwistBsmart, Fries)",(iTwistB),0.073,0.082,0.073,0.994,12.168,0.067,165.279
12,(mMiniTwistBsmart),(iTwistB),0.075,0.082,0.075,0.993,12.150,0.068,130.600
13,(iTwistB),(mMiniTwistBsmart),0.082,0.075,0.075,0.913,12.150,0.068,10.605
154,"(DrumstickKent., ex_45BitesAddon)","(app_bucketfor1, Fries, 2HotWings)",0.071,0.083,0.070,0.986,11.935,0.065,64.856
134,"(app_bucketfor1, Fries)","(DrumstickKent., ex_45BitesAddon)",0.083,0.071,0.070,0.853,11.935,0.065,6.318


### Conclusion
- Looks like association analysis gives blurry image of the situation due to the obvious patterns that are very frequent:
        - Bsmart: Fries, some main (longer, 2strips, iTwistB)
        - app_bucketfor1: ex_45BitesAddon, app_bucketfor1, 2HotWings, DrumstickKent., fries, 
        - etc
- Solution could be in removing such obvious product combinations from the data by hand

# Further cleaning

### Filtering data
We will fillter out less popular products and those which are no actual products.

In [43]:
limit = 1000
val_counts = data['Product'].value_counts()
less_popular_products = list(val_counts[val_counts < limit].index)
val_to_filter = ['Customer'] + less_popular_products

In [45]:
len(val_counts)

585

In [44]:
data_prod_filtered = data[~data['Product'].isin(val_to_filter)]

In [ ]:
# Standard sets
{
'Bsmart': ['mStripsBsmart', 'mMiniTwistBsmart', 'mLongerBsmart', 'Fries', '2Strips', 'Longer', 'iTwistB']
'app_bucketfor1': ['DrumstickKent.','2HotWings','ex_45BitesAddon', 'Fries']
    
}